In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from transformers import pipeline
import pandas as pd

In [ ]:
# Replace this with your own checkpoint
model_checkpoint2 = "aditi2212/roberta-finetuned-ChennaiQA-final"
question_answerer = pipeline("question-answering", model=model_checkpoint2)

In [ ]:
df=pd.read_csv(r'/content/Inference.csv')

In [ ]:
roberta_answers = []

In [ ]:
df_train=pd.read_csv(r'/content/Train_data_final.csv')
df_test=pd.read_csv(r'/content/Test_data_final.csv')
frames = [df_train, df_test]
df_total = pd.concat(frames)

unique_context = pd.DataFrame(df_total['review'].unique())
unique_context.rename( columns={0:'Context'}, inplace=True )
contexts_list = unique_context['Context'].tolist()

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer('all-mpnet-base-v2')
def pickRelevantContext(question, contexts_list):
    question_embedding = model.encode(question)
    context_embeddings = model.encode(contexts_list)
    similarity_scores = cosine_similarity([question_embedding], context_embeddings)
    most_similar_index = similarity_scores.argmax()
    #print("Most relevant context:", contexts_list[most_similar_index])
    #print("Relevance score:", similarity_scores[0][most_similar_index])
    context=contexts_list[most_similar_index]
    return context

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
for index, row in df.iterrows():
    # Extract the question from the current row
    question = row['question']
    context=pickRelevantContext(question, contexts_list)
    # Use RoBERTa to answer the question
    roberta_answer =question_answerer(question=question, context=context)['answer']
    roberta_answers.append(roberta_answer)


df['RoBERTa Answer'] = roberta_answers


In [ ]:
df

,question,Expected Answer,RoBERTa Answer
0,What is Chennai formerly known as?,Madras,Madras
1,What is the capital city of Tamil Nadu?,Chennai,Chennai
2,What is Chennai known for?,"Its rich history, vibrant culture, and archite...","rich history, vibrant culture, and architectur..."
3,In which region of India is Chennai located?,South India,South India
4,What are some of the key features of Chennai?,"Cultural, economic, and educational center","cultural, economic, and educational center"
...,...,...,...
315,What climate does mahabalipuram have?,tropical wet and dry climate,tropical wet and dry climate
316,What is the average temperature at mahabalipuram?,28.4 °C,28.4 °C
317,how many monuments are there in mahabalipuram?,40 ancient monuments,40 ancient monuments
318,How far is shore temple from chennai?,60 kilometres south of Chennai,60 kilometres south of Chennai


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

original_texts = df['Expected Answer'].tolist()
llm_generated_texts = df['RoBERTa Answer'].tolist()

# Tokenize and vectorize the texts
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(original_texts + llm_generated_texts)

# Compute cosine similarity
cosine_similarities = cosine_similarity(tfidf_matrix[:len(original_texts)], tfidf_matrix[len(original_texts):])

# Print cosine similarities
for i, sim in enumerate(cosine_similarities):
    #print(f"Cosine Similarity between text {i+1} and LLM-generated text: {sim[0]}")
    df.loc[i, 'cosine_similarity'] = sim[0]


In [ ]:
df

,question,Expected Answer,RoBERTa Answer,cosine_similarity,jaccard_similarity,levenshtein_distance,normalized_levenshtein,average_similarity,percentage_difference,Semantic_similarity
0,What is Chennai formerly known as?,Madras,Madras,1.0,1.0,0,1.0,0.75,25.0,0.0
1,What is the capital city of Tamil Nadu?,Chennai,Chennai,0.0,1.0,0,1.0,0.5,50.0,0.0
2,What is Chennai known for?,"Its rich history, vibrant culture, and archite...","rich history, vibrant culture, and architectur...",0.0,0.875,4,0.963964,0.459741,54.025901,0.0
3,In which region of India is Chennai located?,South India,South India,0.0,1.0,0,1.0,0.5,50.0,0.0
4,What are some of the key features of Chennai?,"Cultural, economic, and educational center","cultural, economic, and educational center",0.0,0.666667,1,0.990991,0.414414,58.558559,0.0
...,...,...,...,...,...,...,...,...,...,...
315,What climate does mahabalipuram have?,tropical wet and dry climate,tropical wet and dry climate,0.0,1.0,0,1.0,0.5,50.0,0.0
316,What is the average temperature at mahabalipuram?,28.4 °C,28.4 °C,0.0,1.0,0,1.0,0.5,50.0,0.0
317,how many monuments are there in mahabalipuram?,40 ancient monuments,40 ancient monuments,0.0,1.0,0,1.0,0.5,50.0,0.0
318,How far is shore temple from chennai?,60 kilometres south of Chennai,60 kilometres south of Chennai,0.0,1.0,0,1.0,0.5,50.0,0.0


In [ ]:
!pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 14.5 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import jaccard_score
import Levenshtein
# Add columns for Jaccard similarity and Levenshtein distance
df['jaccard_similarity'] = None
df['levenshtein_distance'] = None

# Compute Jaccard similarity and Levenshtein distance
for i, (text1, text2) in enumerate(zip(original_texts, llm_generated_texts)):
    # Tokenize texts
    tokens1 = set(text1.split())
    tokens2 = set(text2.split())

    # Compute Jaccard similarity
    jaccard_similarity = len(tokens1.intersection(tokens2)) / len(tokens1.union(tokens2))
    df.at[i, 'jaccard_similarity'] = jaccard_similarity

    # Compute Levenshtein distance
    levenshtein_distance = Levenshtein.distance(text1, text2)
    df.at[i, 'levenshtein_distance'] = levenshtein_distance

In [ ]:
df

,question,Expected Answer,RoBERTa Answer,cosine_similarity,jaccard_similarity,levenshtein_distance
0,What is Chennai formerly known as?,Madras,Madras,1.0,1.0,0
1,What is the capital city of Tamil Nadu?,Chennai,Chennai,0.0,1.0,0
2,What is Chennai known for?,"Its rich history, vibrant culture, and archite...","rich history, vibrant culture, and architectur...",0.0,0.875,4
3,In which region of India is Chennai located?,South India,South India,0.0,1.0,0
4,What are some of the key features of Chennai?,"Cultural, economic, and educational center","cultural, economic, and educational center",0.0,0.666667,1
...,...,...,...,...,...,...
315,What climate does mahabalipuram have?,tropical wet and dry climate,tropical wet and dry climate,0.0,1.0,0
316,What is the average temperature at mahabalipuram?,28.4 °C,28.4 °C,0.0,1.0,0
317,how many monuments are there in mahabalipuram?,40 ancient monuments,40 ancient monuments,0.0,1.0,0
318,How far is shore temple from chennai?,60 kilometres south of Chennai,60 kilometres south of Chennai,0.0,1.0,0


In [ ]:
import gensim.downloader as api
# Load pre-trained GloVe embeddings
word_vectors = api.load("glove-wiki-gigaword-100")
import numpy as np


#original_texts = df['TS_Content'].tolist()
#llm_generated_texts = df['Generated_Ts_content'].tolist()

# Function to compute sentence embeddings
def sentence_embedding(sentence):
    words = sentence.split()
    vectors = [word_vectors[word] for word in words if word in word_vectors]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)  # Return zeros if no word found in embeddings

# Compute sentence embeddings for original and LLM-generated texts
original_embeddings = np.array([sentence_embedding(text) for text in original_texts])
llm_generated_embeddings = np.array([sentence_embedding(text) for text in llm_generated_texts])

# Compute cosine similarity between corresponding pairs of embeddings
cosine_similarities = cosine_similarity(original_embeddings, llm_generated_embeddings)

# Print cosine similarities
for i, sim in enumerate(cosine_similarities):
    #print(f"Semantic Similarity between text {i+1} and LLM-generated text: {sim[0]}")
    df.loc[i, 'Semantic_similarity'] = sim[0]

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
max_levenshtein_distance = max(df['levenshtein_distance'])
df['normalized_levenshtein'] = 1 - (df['levenshtein_distance'] / max_levenshtein_distance)
df['average_similarity'] = df[['cosine_similarity', 'jaccard_similarity', 'Semantic_similarity', 'normalized_levenshtein']].mean(axis=1)

# Convert the average back to a percentage
df['percentage_difference'] = 100 - (df['average_similarity'] * 100)


In [ ]:
df

,question,Expected Answer,RoBERTa Answer,cosine_similarity,jaccard_similarity,levenshtein_distance,normalized_levenshtein,average_similarity,percentage_difference,Semantic_similarity
0,What is Chennai formerly known as?,Madras,Madras,1.0,1.0,0,1.0,0.75,25.0,0.0
1,What is the capital city of Tamil Nadu?,Chennai,Chennai,0.0,1.0,0,1.0,0.5,50.0,0.0
2,What is Chennai known for?,"Its rich history, vibrant culture, and archite...","rich history, vibrant culture, and architectur...",0.0,0.875,4,0.963964,0.459741,54.025901,0.0
3,In which region of India is Chennai located?,South India,South India,0.0,1.0,0,1.0,0.5,50.0,0.0
4,What are some of the key features of Chennai?,"Cultural, economic, and educational center","cultural, economic, and educational center",0.0,0.666667,1,0.990991,0.414414,58.558559,0.0
...,...,...,...,...,...,...,...,...,...,...
315,What climate does mahabalipuram have?,tropical wet and dry climate,tropical wet and dry climate,0.0,1.0,0,1.0,0.5,50.0,0.0
316,What is the average temperature at mahabalipuram?,28.4 °C,28.4 °C,0.0,1.0,0,1.0,0.5,50.0,0.0
317,how many monuments are there in mahabalipuram?,40 ancient monuments,40 ancient monuments,0.0,1.0,0,1.0,0.5,50.0,0.0
318,How far is shore temple from chennai?,60 kilometres south of Chennai,60 kilometres south of Chennai,0.0,1.0,0,1.0,0.5,50.0,0.0


In [ ]:
df.to_csv('\content\Roberta_answer_similarity.csv')